In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# read in the features
with open('ACCEDEfeaturesArousal_TAC2015.txt') as file:
    lines = file.readlines()
    data = [line.split('\t') for line in lines]

In [3]:
# make a dataframe from it
a = np.asarray(data)
index = a[1:,0]
col = a[0,:]
a = a[1:,:]
df = pd.DataFrame(data=a, index=index, columns=col)
df.head()

,id,name,colorfulness,minEnergy,alpha,lightning,globalActivity,nbWhiteFrames,nbSceneCuts,cutLength,...,wtf_max2stdratio_5,wtf_max2stdratio_6,wtf_max2stdratio_7,wtf_max2stdratio_8,wtf_max2stdratio_9,wtf_max2stdratio_10,wtf_max2stdratio_11,wtf_max2stdratio_12,medianLightness,slope
0,0,ACCEDE00000.mp4,47.939117,0.540554,5,1.057049,0.100531,0,1,117,...,4.00E+00,3.27E+00,2.37E+00,1.69E+00,1.46E+00,1.42E+00,1.41E+00,4.32E+15,0.431373,1.414934\n
1,1,ACCEDE00001.mp4,52.748894,0.798568,41,0.464044,0.062712,0,1,153,...,4.32E+00,3.76E+00,2.91E+00,2.03E+00,1.57E+00,1.43E+00,1.41E+00,3.53E+15,0.219608,1.354866\n
2,2,ACCEDE00002.mp4,60.224403,0.833286,50,0.294487,0.322534,0,3,52,...,4.13E+00,3.31E+00,2.77E+00,2.19E+00,1.54E+00,1.42E+00,1.41E+00,2.42E+15,0.4,1.312889\n
3,3,ACCEDE00003.mp4,82.669434,1.017838,43,0.447583,0.70555,0,1,52,...,3.86E+00,3.20E+00,2.74E+00,2.39E+00,1.74E+00,1.44E+00,1.41E+00,2.09E+15,0.521569,1.334575\n
4,4,ACCEDE00004.mp4,64.427177,0.670391,38,0.059518,0.127405,0,1,117,...,3.74E+00,3.15E+00,2.82E+00,2.38E+00,1.68E+00,1.43E+00,1.41E+00,1.97E+15,0.47451,1.390362\n


In [4]:
# read the annotation
with open('annotations/ACCEDEranking.txt') as file:
    lines = file.readlines()
    annotate = [line.split('\t') for line in lines]

In [5]:
b = np.asarray(annotate)
index = b[1:,0]
col = b[0,:]
b = b[1:,:]
df_annotate = pd.DataFrame(data=b, index=index, columns=col)
df_annotate = df_annotate.drop(['id','name'],axis=1)

In [6]:
total_df = pd.concat([df, df_annotate], axis=1, join='outer')

In [34]:
train_data = np.asarray(df[['colorfulness','minEnergy','alpha','lightning','globalActivity'
                            ,'nbWhiteFrames','nbSceneCuts','cutLength','flatness_env'
                            ,'wtf_max2stdratio_1','wtf_max2stdratio_2','wtf_max2stdratio_3'
                            ,'wtf_max2stdratio_4','wtf_max2stdratio_5','wtf_max2stdratio_6'
                            ,'wtf_max2stdratio_7','wtf_max2stdratio_8','wtf_max2stdratio_9'
                            ,'wtf_max2stdratio_10','wtf_max2stdratio_11','wtf_max2stdratio_12','medianLightness']])
valence = df_annotate['valenceValue'].values
label_valence = [float(v) for v in valence]
arousal = df_annotate['arousalValue'].values
label_arousal = [float(v) for v in arousal]

In [52]:
n = len(train_data)
feat = len(train_data[0])
train = [float(t) for i in range(n) for t in train_data[i] ]
X_train = np.asarray(train).reshape((n,feat))

# normalize the data
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)

In [58]:
# do a quick prediction on the values
# This is the training error

from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train_transformed, label_valence)
pred = reg.predict(X_train_transformed)
error = np.mean((pred-label_valence)**2)
print('Valence MSE:', error)
reg.fit(X_train_transformed, label_arousal)
pred = reg.predict(X_train_transformed)
error = np.mean((pred-label_arousal)**2)
print('Arousal MSE:', error)

Valence MSE: 0.37336196785071085
Arousal MSE: 0.8423003632942875


In [55]:
# tyy PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
X_new = pca.fit_transform(X_train_transformed)
reg.fit(X_new, label_valence)
pred = reg.predict(X_new)
error = np.sqrt(np.mean((pred-label_valence)**2))
print('Valence RMSE:', error)

Valence RMSE: 0.6139517279715563


In [51]:
# cross-validation
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn import metrics
scorer = metrics.make_scorer(metrics.mean_squared_error)
clf = make_pipeline(preprocessing.StandardScaler(), linear_model.LinearRegression())
cross_val_score(clf, train, label_valence, cv=10, scoring=scorer)

array([0.41147092, 0.38578348, 0.47036316, 0.38328823, 0.36913974,
       0.37329604, 0.33224317, 0.44994844, 0.3440702 , 0.3073722 ])